# Evaluation

## Objectif

L'objectif de cette évaluation est de parvenir à détecter si une requête est une tentative d'intrusion ou bien une requête légitime.

Votre modèle devrait obtenir un score de 95%.

## Livrable

Vous devrez compléter ce notebook Jupyter et répondre aux questions du Google Form associé [EPSI - PARIS - 2026 - TRDE707 - TP Noté](https://forms.gle/ZhcULFdgYDDm4P7Q9).

## Dépendances et modules

Les modules disponibles pour l'exécution de ce notebook sont :
* pandas
* scikit-learn
* matplotlib

## Source de données

Les sources de données sont des fichiers au format CSV. Vous pouvez les télécharger depuis le dossier [trde707-datasets](https://drive.google.com/drive/folders/135R4uXKxgwFHxNQY8x3iVmSrfGWmYM8a?usp=sharing).

L'archive public_network_log.zip contient un unique fichier public_network_log.csv qui représente un log de requêtes identifiées comme une tentative d'intrusion ou une requête légitime.

L'archive dbip-country-lite-2026-01.zip contient un unique fichier dbip-country-lite-2026-01.csv qui associe à chaque plage d'IP un code Pays. Cette base de données est issue du portail [dbip](https://db-ip.com/db/format/ip-to-country/csv.html).


Charger les données


Afficher les effectifs de la colonne `Intrusion`.


Afficher les 2 représentations les plus adaptées à la colonne `Payload_Size`.


Afficher la représentation la plus adaptée à `Port`.


Afficher sur un même graphique les variables `Payload_Size`, `Port` et `Intrusion`.


Sélectionner les colonnes pour lesquelles un encodage est nécessaire.


Sélectionner les colonnes pour lesquelles une standardisation est nécessaire.


Sélectionner un modèle et l'entrainer.

Evaluer les performances de votre modèle.

Choisir un affichage permettant de restituer les performances du modèle sélectionné.